# 🎬 VideoRobot — Full Automatic Pipeline (Colab Runner)
✅ Clone → Install → Start Backend → Healthcheck → Smart Manifest → Render → Download → Copy to Drive

**Features:**
- Works fully inside Google Colab (no desktop needed)
- Pulls assets from Drive or `/content/Assets`
- Renders final MP4 and stores to `/Output` or Drive
- Clean logs, auto port detection, and self-healing backend

🔹 Compatible with: `https://github.com/englishpodcasteasy-glitch/videorobot`

---

In [ ]:
# -*- coding: utf-8 -*-
"""
VideoRobot Full Pipeline — Colab Notebook Version
Mount Drive → Clone → Install → Start Backend → Healthcheck → Build Manifest → Render → Monitor → Download → Copy
"""

# 📦 Imports & constants
import os, pathlib, shutil, subprocess, sys, time, socket, signal, tempfile, json, requests
from IPython.display import HTML, display

REPO_URL = "https://github.com/englishpodcasteasy-glitch/videorobot.git"
CLONE_DIR = pathlib.Path("/content/videorobot")
BACKEND_DIR = CLONE_DIR / "backend"
MANIFEST_PATH = pathlib.Path("/content/manifest.json")
LOG_PATH = pathlib.Path("/content/backend.log")
DEFAULT_PORTS = [8000, 8001, 8002, 7861, 9000]
HEALTH_ENDPOINTS = ["/healthz", "/health", "/version", "/"]
POLL_INTERVAL = 1.0
POLL_TIMEOUT = 15 * 60

DRIVE_ROOT = ASSETS = OUTPUT = None

## 🧠 1️⃣ Mount Google Drive & prepare workspace

In [ ]:
def mount_drive_and_prepare():
    global DRIVE_ROOT, ASSETS, OUTPUT
    try:
        from google.colab import drive  # type: ignore
        drive.mount('/content/drive')
        DRIVE_ROOT = pathlib.Path('/content/drive/MyDrive/VideoRobot')
        ASSETS = DRIVE_ROOT / 'Assets'
        OUTPUT = DRIVE_ROOT / 'Output'
        ASSETS.mkdir(parents=True, exist_ok=True)
        OUTPUT.mkdir(parents=True, exist_ok=True)
        print('✅ Drive mounted at:', DRIVE_ROOT)
        print('📂 Assets:', ASSETS)
        print('📂 Output:', OUTPUT)
    except Exception as e:
        print('ℹ️ Drive mount failed:', e)
        ASSETS = pathlib.Path('/content/Assets'); OUTPUT = pathlib.Path('/content/Output')
        ASSETS.mkdir(exist_ok=True); OUTPUT.mkdir(exist_ok=True)
        print('📂 Using local folders:', ASSETS, OUTPUT)
    os.environ['PYTHONUNBUFFERED'] = '1'
    os.environ['SDL_AUDIODRIVER'] = 'dummy'
    os.environ['XDG_RUNTIME_DIR'] = '/tmp'
mount_drive_and_prepare()

## ⚙️ 2️⃣ Clone repo + install dependencies

In [ ]:
def clone_and_install():
    shutil.rmtree(CLONE_DIR, ignore_errors=True)
    print('▶ git clone:', REPO_URL)
    subprocess.run(['git','clone','--depth','1',REPO_URL,str(CLONE_DIR)], check=True)
    print('✅ Repo cloned')

    subprocess.run(['apt-get','-qq','update'], check=True)
    subprocess.run(['apt-get','-qq','-y','install','ffmpeg'], check=True)
    print('✅ ffmpeg installed')

    subprocess.run([sys.executable,'-m','pip','install','-q','--upgrade','pip','setuptools','wheel'], check=True)
    req_file = BACKEND_DIR / 'requirements.txt'
    if req_file.exists():
        subprocess.run([sys.executable,'-m','pip','install','-q','-r',str(req_file)], check=True)
    else:
        subprocess.run([sys.executable,'-m','pip','install','-q','flask','flask-cors','moviepy','imageio-ffmpeg','requests'], check=True)
    subprocess.run([sys.executable,'-m','pip','install','-q','moviepy>=1.0.3,<3','imageio-ffmpeg>=0.4.9,<1'], check=True)
    print('✅ Dependencies installed')

clone_and_install()

## 🚀 3️⃣ Start backend & run healthcheck

In [ ]:
def start_backend_and_healthcheck():
    assert BACKEND_DIR.exists(), 'backend folder missing'
    initf = BACKEND_DIR / '__init__.py'
    if not initf.exists(): initf.write_text('')
    sys.path.insert(0, str(CLONE_DIR))
    
    import re
    patched = []
    for py in BACKEND_DIR.rglob('*.py'):
        txt = py.read_text(errors='ignore'); orig = txt
        txt = re.sub(r'\bfrom\s+config\s+import\b','from .config import',txt)
        txt = re.sub(r'\bfrom\s+scheduler\s+import\b','from .scheduler import',txt)
        txt = re.sub(r'\bfrom\s+utils\s+import\b','from .utils import',txt)
        txt = re.sub(r'\bfrom\s+renderer(_service)?\s+import\b',r'from .renderer\1 import',txt)
        if txt!=orig: py.write_text(txt); patched.append(py.name)
    print('🛠 patched imports:', patched if patched else '(none)')

    for pat in ('backend/main.py','backend.main'):
        for pid in subprocess.getoutput(f"pgrep -f '{pat}' || true").split():
            try: os.kill(int(pid), signal.SIGKILL)
            except: pass

    def pick_port():
        for p in DEFAULT_PORTS:
            try:
                with socket.socket() as s: s.bind(('127.0.0.1',p)); return p
            except OSError: continue
        return 8000

    port = pick_port(); os.environ['BACKEND_PORT']=str(port)
    base=f'http://127.0.0.1:{port}'
    print('🚪 Port =',port)

    env=os.environ.copy()
    subprocess.Popen([sys.executable,'-m','backend.main'],cwd=str(CLONE_DIR),
                     stdout=open(LOG_PATH,'w'),stderr=subprocess.STDOUT,text=True,env=env)

    ok=False
    for delay in (1,2,4,8,8,8,8):
        for ep in HEALTH_ENDPOINTS:
            try:
                r=requests.get(base+ep,timeout=2)
                if r.ok: print('✅ Health via',ep,'→',(r.text or '')[:160]); ok=True; break
            except: pass
        if ok: break; time.sleep(delay)
    if not ok:
        print('❌ backend not healthy. tail log:')
        print(subprocess.getoutput(f'tail -n 150 {LOG_PATH}'))
        raise SystemExit('Backend failed')
    print('✅ Backend ready at',base)
    return base

BASE=start_backend_and_healthcheck()

## 🧾 4️⃣ Auto-build manifest (auto-detects intro/outro/audio/BGM)

In [ ]:
def build_manifest():
    def pick(pats):
        for p in pats:
            files=list(ASSETS.glob(p));
            if files: return files[0]
        return None

    voice=pick(['*.mp3','*.wav','*.m4a']); bgvid=pick(['*Background*.mp4','*.mp4']); bgimg=pick(['*Background*.png','*.jpg']);
    intro=pick(['*Intro*.mp4']); outro=pick(['*Outro*.mp4']); cta=pick(['*CTA*.mp4']); bgm=pick(['*music*.mp3']); broll=pick(['*broll*.mp4']);

    tracks=[]
    if intro: tracks.append({"type":"video","src":str(intro),"start":0})
    if bgvid: tracks.append({"type":"video","src":str(bgvid),"start":0})
    elif bgimg: tracks.append({"type":"image","src":str(bgimg),"start":0,"duration":60})
    if voice: tracks.append({"type":"audio","src":str(voice),"start":0})
    if broll: tracks.append({"type":"video","src":str(broll),"start":6,"duration":4})
    tracks.append({"type":"text","content":"Real Smart English","start":0.6,"duration":3.5,"x":80,"y":140,"size":84,"color":"#FFD700"})
    if cta: tracks.append({"type":"video","src":str(cta),"start":12})
    if outro: tracks.append({"type":"video","src":str(outro),"start":15})

    manifest={
      "seed":11,
      "video":{"width":1080,"height":1920,"fps":30},
      "tracks":tracks,
      "config":{
        "aspectRatio":"9:16",
        "audio":{"useVAD":True,"targetLufs":-16.0},
        "bgm":({"path":str(bgm),"gain_db":-8,"ducking":True} if bgm else {})
      }
    }
    MANIFEST_PATH.write_text(json.dumps(manifest,ensure_ascii=False,indent=2))
    print('✅ Manifest saved →',MANIFEST_PATH)
    return manifest

build_manifest()

## 🎥 5️⃣ Render video + track progress + download

In [ ]:
def render_and_retrieve():
    payload=json.loads(MANIFEST_PATH.read_text())
    r=requests.post(BASE+'/render',json=payload,timeout=180)
    r.raise_for_status(); js=r.json()
    job=js.get('job_id') or js.get('data',{}).get('job_id') or js.get('jobId'); assert job
    print('🎬 job_id:',job)

    end=time.time()+POLL_TIMEOUT; t=0
    while time.time()<end:
        g=requests.get(BASE+f'/progress/{job}',timeout=10)
        if g.ok:
            pj=g.json(); st=pj.get('state') or pj.get('data',{}).get('state'); pct=pj.get('pct') or pj.get('data',{}).get('pct');
            print(f'tick {t:03d}:',st,pct)
            if st=='success': break
            if st=='error': raise SystemExit(pj)
        time.sleep(POLL_INTERVAL); t+=1

    out=pathlib.Path(f'/content/{job}.mp4')
    url=BASE+f'/download?jobId={job}'
    tmp=pathlib.Path(tempfile.NamedTemporaryFile(delete=False,suffix='.mp4').name)
    with requests.get(url,stream=True) as d:
        d.raise_for_status()
        with open(tmp,'wb') as f:
            for chunk in d.iter_content(8192): f.write(chunk)
    shutil.move(str(tmp),str(out))
    print('📦 Downloaded',out)
    try:
        dst=OUTPUT/out.name; shutil.copy2(out,dst); print('✅ Copied to Drive:',dst)
    except Exception as e: print('ℹ️ Copy skipped:',e)
    display(HTML(f'<a href="{url}" target="_blank"><button style="font-size:16px;padding:10px;background:#4CAF50;color:white;border:none;border-radius:6px;">🔗 Download MP4</button></a>'))
    print('🎉 Done!')

render_and_retrieve()